In [1]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
import os
import torch
from torch import nn
from torch.utils.data import Dataset as TorchDataset
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
from torchinfo import summary as torch_summary
import matplotlib.pyplot as plt
from PIL import Image, ImageFile
import math
import time
import wandb

In [2]:
@dataclass
class Config:
    train_csv_filepath: str
    test_csv_filepath: str
    submission_filepath: str
    images_root_folder: str
    training_output_folder: str
    saved_weights_filepath: str
    device: str

    def __post_init(self):
        """ For configuration variables that are shared across environments """
        self.num_classes = 12
        self.batch_size = 32
        self.starting_learning_rate = 4e-4
        self.max_epochs = 100
        self.seed = 1234
        ...

    # noinspection PyAttributeOutsideInit
    def init(self, training):
        """ Adjust configuration setup for training vs inference """
        self.training = training

        if self.training:
            os.makedirs(self.training_output_folder, exist_ok=True)

        self.imagenet_mean_cpu_tensor = torch.tensor(imagenet_mean_array)
        self.imagenet_std_cpu_tensor = torch.tensor(imagenet_std_array)
        self.channelwise_imagenet_mean_cpu_tensor = self.imagenet_mean_cpu_tensor.view(3, 1, 1)
        self.channelwise_imagenet_std_cpu_tensor = self.imagenet_std_cpu_tensor.view(3, 1, 1)
        self.imagenet_mean_gpu_tensor = gpu_tensor(imagenet_mean_array)
        self.imagenet_std_gpu_tensor = gpu_tensor(imagenet_std_array)
        self.channelwise_imagenet_mean_gpu_tensor = self.imagenet_mean_gpu_tensor.view(3, 1, 1)
        self.channelwise_imagenet_std_gpu_tensor = self.imagenet_std_gpu_tensor.view(3, 1, 1)

        ...

config: Config = None
""" Set to environment-relevant config before training/inference """;

In [3]:
local_config = Config(
    train_csv_filepath='data/train.csv',
    test_csv_filepath='data/test.csv',
    submission_filepath='data/submission.csv',
    images_root_folder='data/images',
    training_output_folder='data_gen/training_output/',
    saved_weights_filepath='data_gen/training_output/model_weights.pth',
    device='cpu',
)
kaggle_config = Config(
    train_csv_filepath='TODO',
    test_csv_filepath='TODO',
    images_root_folder='TODO',
    submission_filepath='TODO',
    training_output_folder='TODO',
    saved_weights_filepath='TODO',
    device='cuda',
)

In [4]:
imagenet_mean_array = np.array([0.485, 0.456, 0.406], dtype=np.float32)
imagenet_std_array = np.array([0.229, 0.224, 0.225], dtype=np.float32)

def gpu_tensor(numpy_array):
    return torch.tensor(numpy_array, device=config.device)

def visualize_image(image_tensor):
    """ Input tensor should be on gpu """
    image = denormalize(image_tensor, config.channelwise_imagenet_mean_gpu_tensor, config.channelwise_imagenet_std_gpu_tensor)
    image = torch.clamp(image, 0, 1)
    image = image.permute(1, 2, 0).cpu().numpy()
    image = (image * 255).astype('uint8')
    plt.imshow(image)
    plt.axis('off')
    plt.show()

def normalize(tensor, mean, std):
    return (tensor - mean) / std

def denormalize(tensor, mean, std):
    return tensor * std + mean

def load_pil_image_from_id(image_id) -> ImageFile.ImageFile:
    return Image.open(config.images_root_folder + image_id + '.jpg')

In [5]:
@dataclass
class ImageSegmentationDataset(TorchDataset):
    image_ids: np.ndarray
    masks_image_ids: np.ndarray

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image = load_pil_image_from_id(self.image_ids[idx])
        mask = load_pil_image_from_id(self.masks_image_ids[idx])
        return image, mask

In [6]:
# test_model = deeplabv3_resnet50(weights=DeepLabV3_ResNet50_Weights.DEFAULT)
# torch_summary(test_model)

In [7]:
# test_model

In [8]:
# aux_classifier_children = test_model.aux_classifier.children()
# for layer in aux_classifier_children:
#     print(layer)

In [9]:
def create_deeplab_v3_model() -> nn.Module:
    weights = DeepLabV3_ResNet50_Weights.DEFAULT if config.training else None
    model = deeplabv3_resnet50(weights=weights, num_classes=config.num_classes, aux_loss=True)

    for param in model.backbone.parameters():
        param.requires_grad = False

    for param in model.classifier.parameters():
        param.requires_grad = True
    for param in model.aux_classifier.parameters():
        param.requires_grad = True

    model.to(config.device)

    return model

In [9]:
def dice_coefficient_score(pred_masks, true_masks):
    intersection = (pred_masks * true_masks).sum(dim=(1, 2))
    union = pred_masks.sum(dim=(1, 2)) + true_masks.sum(dim=(1, 2))
    return (2 * intersection) / union

def train_one_epoch(start_time, model, loader, optimizer, loss_function):
    model.train()
    running_loss = 0.0

    all_pred_masks = []
    all_true_masks = []

    num_batches = math.ceil(len(loader.dataset) / config.batch_size)
    for batch_number, (x, y) in enumerate(loader):
        print(f't={time.time() - start_time:.2f}: Loading training batch {batch_number + 1}/{num_batches}')

        x = x.to(config.device, non_blocking=True)
        y = y.to(config.device, non_blocking=True)

        if batch_number == 0:
            allocated = torch.cude.memory_allocated(config.device) / 1024**3
            reserved = torch.cude.memory_reserved(config.device) / 1024**3
            print(f'Memory allocated={allocated:.2f} GiB, reserved={reserved:.2f} GiB')
            print(f'First image:')
            visualize_image(x[0])
            print(f'First mask:')
            visualize_image(y[0])

        optimizer.zero_grad()
        preds = model(x)
        loss = loss_function(preds, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

        all_pred_masks.append(preds.detach().cpu())
        all_true_masks.append(y.detach().cpu())

    epoch_loss = running_loss / len(loader.dataset)

    all_pred_masks = torch.cat(all_pred_masks, dim=0).numpy()
    all_true_masks = torch.cat(all_true_masks, dim=0).numpy()

    epoch_score = dice_coefficient_score(all_pred_masks, all_true_masks)

    return epoch_loss, epoch_score

@torch.no_grad()
def validate_one_epoch(start_time, model, loader, loss_function):
    model.eval()
    running_loss = 0.0

    all_pred_masks = []
    all_true_masks = []
    num_batches = math.ceil(len(loader.dataset) / config.batch_size)
    for batch_number, (x, y) in enumerate(loader):
        print(f't={time.time() - start_time:.2f}: Loading validation batch {batch_number + 1}/{num_batches}')

        x = x.to(config.device, non_blocking=True)
        y = y.to(config.device, non_blocking=True)

        if batch_number == 0:
            print('First image:')
            visualize_image(x[0])
            print('First mask:')
            visualize_image(y[0])

        preds = model(x)
        loss = loss_function(preds, y)

        running_loss += loss.item() * x.size(0)

        all_pred_masks.append(preds.detach().cpu())
        all_true_masks.append(y.detach().cpu())

    epoch_loss = running_loss / len(loader.dataset)

    all_pred_masks = torch.cat(all_pred_masks, dim=0).numpy()
    all_true_masks = torch.cat(all_true_masks, dim=0).numpy()

    epoch_score = dice_coefficient_score(all_pred_masks, all_true_masks)

    return epoch_loss, epoch_score

In [ ]:
def train():
    config.init(training=True)

    start_time = time.time()
    print('t=0: Starting data prep and model loading')

    run = wandb.init(
        project='drone_image_segmentation',
        name=f'run={int(start_time)}',
        config={
            'batch_size': config.batch_size,
            'learning_rate': config.starting_learning_rate,
            'max_epochs': config.max_epochs,
            'seed': config.seed,
            'model': 'deeplabv3_resnet50',
            'optimizer': 'Adam',
        },
    )

    train_df = pd.read_csv(config.train_csv_filepath)
    ...